# <ins> Futuristic Airbnb </ins>

Airbnb has successfully disrupted the traditional hospitality industry as more and more travelers decide to use Airbnb as their primary accommodation provider. Since its beginning in 2008, Airbnb has seen an enormous growth, with the number of rentals listed on its website growing exponentially each year. In Germany, no city is more popular than Berlin. That implies that Berlin is one of the hottest markets for Airbnb in Europe, with over 22,552 listings as of November 2018.Here we will be using Python with some packages like Pandas and Matplotlib to analyze data  to find a better Accommodation based on customer satisfaction and to improve the Airbnb business based on certain factors in Berlin.

<hr> </hr>

In [10]:
#Importing Dependencies
import pandas as pd
import numpy as np

In [15]:
#Reading Files and storing in Dataframes

Calendar_Summary_df = pd.read_csv("../Dataset/calendar_summary.csv")
Listings_df = pd.read_csv("../Dataset/listings.csv")
Listings_Summary_df = pd.read_csv("../Dataset/listings_summary.csv")
Neighbourhoods_df = pd.read_csv("../Dataset/neighbourhoods.csv")
Reviews_df = pd.read_csv("../Dataset/reviews.csv")
Reviews_Summary = pd.read_csv("../Dataset/reviews_summary.csv")


In [18]:
# printing Reviews_summary
Reviews_Summary.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2015,69544350,2016-04-11,7178145,Rahel,Mein Freund und ich hatten gute gemütliche vie...
1,2015,69990732,2016-04-15,41944715,Hannah,Jan was very friendly and welcoming host! The ...
2,2015,71605267,2016-04-26,30048708,Victor,Un appartement tres bien situé dans un quartie...
3,2015,73819566,2016-05-10,63697857,Judy,"It is really nice area, food, park, transport ..."
4,2015,74293504,2016-05-14,10414887,Romina,"Buena ubicación, el departamento no está orden..."


### <ins> Identifying the Columns </ins>

In [17]:
Calendar_Summary_df.columns

Index(['listing_id', 'date', 'available', 'price'], dtype='object')

In [4]:
Listings_df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [5]:
Listings_Summary_df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'access', 'interaction', 'house_rules',
       'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url',
       'host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms',

In [6]:
Neighbourhoods_df.columns

Index(['neighbourhood_group', 'neighbourhood'], dtype='object')

### <ins> Keeping the Necessary Columns </ins>

In [7]:
#Columns which are necessary alone we can store it in the below df. ()

#Calendar_Summary_df = 
#Listings_df = 
#Listings_Summary_df = 
#Neighbourhoods_df = 
#Reviews_df = 
#Reviews_Summary = 

### <ins> Data Cleaning </ins>

In [8]:
#Data Cleaning the Amenities Column

Amenities = Listings_Summary_df.amenities

Amenities_List = []

for x in Amenities:

        x=x.replace('"','')
        x=x.replace('{','')
        x=x.replace('}','')
        Amenities_List += x.split(',')
        
Amenities_New = pd.DataFrame(Amenities_List)

Amenities_New = Amenities_New.rename(columns = {0:"Amenities"})

Amenities_New.head()


,Amenities
0,TV
1,Cable TV
2,Wifi
3,Kitchen
4,Gym


In [9]:
Amenities_Group = Amenities_New.groupby('Amenities')

Amenities_Count = Amenities_Group["Amenities"].count()

Amenities_Count = Amenities_Count.sort_values(ascending=False)

Amenities_Count = Amenities_Count.drop(['translation missing: en.hosting_amenity_50', 'translation missing: en.hosting_amenity_49'],axis=0)

print(Amenities_Count)

Amenities_Count/Amenities_Count.sum()*100 

(Amenities_Count[0:30]/Amenities_Count.sum()*100).sum()

Amenities_Count[0:30]












Amenities
Wifi                          21512
Kitchen                       21332
Heating                       21190
Essentials                    20329
Washer                        18497
Hair dryer                    14891
Laptop friendly workspace     14447
Hangers                       13908
Iron                          11206
Shampoo                       10654
TV                            10134
Hot water                      9744
Family/kid friendly            8141
Internet                       7909
Host greets you                6663
Smoke detector                 6457
Buzzer/wireless intercom       6116
Lock on bedroom door           5294
Refrigerator                   5212
Free street parking            5190
Dishes and silverware          5082
Elevator                       4867
Bed linens                     4842
Cooking basics                 4834
Stove                          4816
Smoking allowed                4589
Oven                           4312
First aid kit     

Amenities
Wifi                         21512
Kitchen                      21332
Heating                      21190
Essentials                   20329
Washer                       18497
Hair dryer                   14891
Laptop friendly workspace    14447
Hangers                      13908
Iron                         11206
Shampoo                      10654
TV                           10134
Hot water                     9744
Family/kid friendly           8141
Internet                      7909
Host greets you               6663
Smoke detector                6457
Buzzer/wireless intercom      6116
Lock on bedroom door          5294
Refrigerator                  5212
Free street parking           5190
Dishes and silverware         5082
Elevator                      4867
Bed linens                    4842
Cooking basics                4834
Stove                         4816
Smoking allowed               4589
Oven                          4312
First aid kit                 4011
Cable TV  